# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

UndefVarError: UndefVarError: model not defined

## 讀取資料

In [2]:
# train_X, train_y = SVHN2.traindata(Float32, 1:20000)
# test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)
train_X, train_y = SVHN2.traindata(Float32, 1:2000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:200)
println(size(train_X))
println(typeof(train_X))
println(size(train_y))
println(typeof(train_y))

(32, 32, 3, 2000)
Array{Float32,4}
(2000,)
Array{Int64,1}


In [3]:
# train_X = reshape(train_X,1024 , 1, :)
# test_X = reshape(test_X, 1024,3, 1, :)
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)
println(size(train_X))
println(typeof(train_X))
println(size(train_y))

(32, 32, 3, 2000)
Array{Float32,4}
(10, 2000)


In [4]:
# https://fluxml.ai/Flux.jl/stable/data/dataloader/

batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

┌ Warning: Number of data points less than batchsize, decreasing the batchsize to 200
└ @ Flux.Data C:\Users\User\.julia\packages\Flux\Fj3bt\src\data\dataloader.jl:64


DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [5]:
# https://fluxml.ai/Flux.jl/stable/models/layers/

# model = # write your model here
# model = Chain(
#     Conv((3, 3), 3=>48, pad=(1,1), relu),
#     MaxPool((2, 2)),
#     Conv((3, 3), 48=>96, pad=(1,1), relu),
#     MaxPool((2, 2)),
#     Conv((3, 3), 96=>192, pad=(1,1), relu),
#     MaxPool((2, 2)),
#     Conv((3, 3), 192=>384, pad=(1,1), relu),
#     MaxPool((2, 2)),
#     Conv((3, 3), 384=>384, pad=(1,1), relu),
#     MaxPool((2, 2)),
#     flatten,
#     Dense(384, 10),
#     softmax)

model = Chain(
    Conv((3, 3), 3=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    flatten,
    Dense(512, 10),
    softmax)

Chain(Conv((3, 3), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(512, 10), softmax)

In [6]:
### 損失函數
loss(x, y) = logitcrossentropy(model(x), y)

### Callback 函式
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

In [7]:
epochs = 20
# epochs = 10
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2960062f0


┌ Info: Epoch 2
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2546573f0


┌ Info: Epoch 3
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 2.221222f0


┌ Info: Epoch 19
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.1935463f0


┌ Info: Epoch 20
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2059414f0


In [8]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
accuracy(test_X, test_y)

0.24

### MLP

In [9]:
model_MLP = Chain(
  Dense(3072, 1536, σ),
  Dense(1536, 768, σ),
  Dense(768, 256, σ),
  Dense(256, 128, σ),
  Dense(128, 10),
  softmax)

epochs = 20
@epochs epochs Flux.train!(loss, params(model_MLP), train, ADAM(0.005), cb=throttle(evalcb, 10))
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
accuracy(test_X, test_y)

┌ Info: Epoch 1
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2081587f0


┌ Info: Epoch 2
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2081587f0


┌ Info: Epoch 3
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

┌ Info: Epoch 19
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2081587f0


┌ Info: Epoch 20
└ @ Main C:\Users\User\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2081587f0


0.24